In [17]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
import operator

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [2]:
data = pd.read_csv('C:/Users/Rhea/Documents/INNOVATION/PROJECT/ALL_REVIEWS.csv',
                   names=['Phone_Name','Date','Rating_Out_of_5','Review_Title','Review_Text'])

In [3]:
data.head()

,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
0,"BLU VIVO XL2 - 5.5"" 4G LTE GSM","February 20, 2017",5.0,ABOUT FREAKING TIME!!!! UPDATED POST!!,"Over the years, I've tried maybe 6 Blu phones...."
1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM","March 7, 2017",5.0,Open Honest Review,"People love to get a new phone and, while rid..."
2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM","May 30, 2017",1.0,Update - don't waste your money! A pretty sol...,Update seven months in: I've had to go from f...
3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM","March 3, 2017",1.0,Spyware still. Come on Blu get it together!,"Still has a security issue,waiting on Blu to r..."
4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM","June 29, 2017",2.0,Absolutely amazing for this price range,The first time it hit the ground the screen cr...


In [4]:
data.shape

(2170, 5)

In [5]:
data['Review_Text'].values[5]

"I have to give this 5 stars! For the price this is a phenomenal deal. I waited 2 weeks to write a review and I must say this is a great phone! Looks good, works really well, storage is terrific. I did download the Evie go Launcher because I didn't like the stock launchers but that is all! I am never spending 700 on a phone anymore as long as BLU makes affordable awesome devices like this vivo xl2."

In [6]:
data_samples=data['Review_Text'].tolist()
data_samples[1]

'People love to get a new phone and,  while riding that new tech high, get on Amazon and shout "BEST PHONE EVER"  when obviously it\'s not.  Here is my open honest list view of the Pros and Cons of this phone.Pros:Price: Even at the suggested retail of 149 usd, this phone out performs the price tagBuild quality: Built like a 700 usd flagship device. Manages to look both beautiful & professionalCall quality :  Perfect, which means it does what a phone is supposed to doScreen: at 720p its NOT as nice as an iPhone or Pixel, but it does look amazing on the 5.5 inch screenAntenna: Most forgotten feature of reviews... This phone has service in the basement of the hospital I work at, on both on Tmobile AND AT&T.  Having duel Sims,  I decided to put in both my personal and work sim cards in the phone. The Sony Xperia M5 Ultra I had before this got zero service for most parts of the basement.  This Vivo XL2 has at least 2 bars of 4g lte service through out the entire basement. Same sim cards, s

In [7]:
data_samples = [str(w).lower() for w in data_samples]

In [8]:
tokenized_docs = [word_tokenize(doc) for doc in data_samples]
print(tokenized_docs[5])

['i', 'have', 'to', 'give', 'this', '5', 'stars', '!', 'for', 'the', 'price', 'this', 'is', 'a', 'phenomenal', 'deal', '.', 'i', 'waited', '2', 'weeks', 'to', 'write', 'a', 'review', 'and', 'i', 'must', 'say', 'this', 'is', 'a', 'great', 'phone', '!', 'looks', 'good', ',', 'works', 'really', 'well', ',', 'storage', 'is', 'terrific', '.', 'i', 'did', 'download', 'the', 'evie', 'go', 'launcher', 'because', 'i', 'did', "n't", 'like', 'the', 'stock', 'launchers', 'but', 'that', 'is', 'all', '!', 'i', 'am', 'never', 'spending', '700', 'on', 'a', 'phone', 'anymore', 'as', 'long', 'as', 'blu', 'makes', 'affordable', 'awesome', 'devices', 'like', 'this', 'vivo', 'xl2', '.']


## Removing punctuation
Punctuation can help with tokenizers, but once you've done that, there's no reason to keep it around. There are tons of ways to remove punctuation. Since we have already learned regex, how would we do this?

In [9]:
print(string.punctuation)
str1=string.punctuation
str1=str1.replace('/','')
print(str1)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&'()*+,-.:;<=>?@[\]^_`{|}~


In [10]:
regex = re.compile('[%s]' % re.escape(str1)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print (tokenized_docs_no_punctuation[5])

['i', 'have', 'to', 'give', 'this', '5', 'stars', 'for', 'the', 'price', 'this', 'is', 'a', 'phenomenal', 'deal', 'i', 'waited', '2', 'weeks', 'to', 'write', 'a', 'review', 'and', 'i', 'must', 'say', 'this', 'is', 'a', 'great', 'phone', 'looks', 'good', 'works', 'really', 'well', 'storage', 'is', 'terrific', 'i', 'did', 'download', 'the', 'evie', 'go', 'launcher', 'because', 'i', 'did', 'nt', 'like', 'the', 'stock', 'launchers', 'but', 'that', 'is', 'all', 'i', 'am', 'never', 'spending', '700', 'on', 'a', 'phone', 'anymore', 'as', 'long', 'as', 'blu', 'makes', 'affordable', 'awesome', 'devices', 'like', 'this', 'vivo', 'xl2']


## Cleaning text of stopwords
There are some really basic words that just don't matter. NLTK comes with a list of them for many languages.

In [11]:
tokenized_docs_no_stopwords = []
for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    tokenized_docs_no_stopwords.append(new_term_vector)
            
print (tokenized_docs_no_stopwords[1])

['people', 'love', 'get', 'new', 'phone', 'riding', 'new', 'tech', 'high', 'get', 'amazon', 'shout', 'best', 'phone', 'ever', 'obviously', 'open', 'honest', 'list', 'view', 'pros', 'cons', 'phonepros', 'price', 'even', 'suggested', 'retail', '149', 'usd', 'phone', 'performs', 'price', 'tagbuild', 'quality', 'built', 'like', '700', 'usd', 'flagship', 'device', 'manages', 'look', 'beautiful', 'professionalcall', 'quality', 'perfect', 'means', 'phone', 'supposed', 'doscreen', '720p', 'nice', 'iphone', 'pixel', 'look', 'amazing', '55', 'inch', 'screenantenna', 'forgotten', 'feature', 'reviews', 'phone', 'service', 'basement', 'hospital', 'work', 'tmobile', 'duel', 'sims', 'decided', 'put', 'personal', 'work', 'sim', 'cards', 'phone', 'sony', 'xperia', 'm5', 'ultra', 'got', 'zero', 'service', 'parts', 'basement', 'vivo', 'xl2', 'least', '2', 'bars', '4g', 'lte', 'service', 'entire', 'basement', 'sim', 'cards', 'radio', 'bands', 'supported', 'yet', 'phone', 'gets', 'signalsound', 'loud', 'cr

### Stemming and Lemmatizing
If you have taken linguistics, you may be familiar with morphology. This is the belief that words have a root form. If you want to get to the basic term meaning of the word, you can try applying a stemmer or lemmatizer. Here are three very popular methods ready to go right out of the NLTK box. It's up to you to see which one you want to use.

In [12]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(wordnet.lemmatize(word))
        #final_doc.append(snowball.stem(word))
        #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
    preprocessed_docs.append(final_doc)

print (preprocessed_docs[1])

['people', 'love', 'get', 'new', 'phone', 'riding', 'new', 'tech', 'high', 'get', 'amazon', 'shout', 'best', 'phone', 'ever', 'obviously', 'open', 'honest', 'list', 'view', 'pro', 'con', 'phonepros', 'price', 'even', 'suggested', 'retail', '149', 'usd', 'phone', 'performs', 'price', 'tagbuild', 'quality', 'built', 'like', '700', 'usd', 'flagship', 'device', 'manages', 'look', 'beautiful', 'professionalcall', 'quality', 'perfect', 'mean', 'phone', 'supposed', 'doscreen', '720p', 'nice', 'iphone', 'pixel', 'look', 'amazing', '55', 'inch', 'screenantenna', 'forgotten', 'feature', 'review', 'phone', 'service', 'basement', 'hospital', 'work', 'tmobile', 'duel', 'sims', 'decided', 'put', 'personal', 'work', 'sim', 'card', 'phone', 'sony', 'xperia', 'm5', 'ultra', 'got', 'zero', 'service', 'part', 'basement', 'vivo', 'xl2', 'least', '2', 'bar', '4g', 'lte', 'service', 'entire', 'basement', 'sim', 'card', 'radio', 'band', 'supported', 'yet', 'phone', 'get', 'signalsound', 'loud', 'crisp', 'eve

In [13]:
all_words=[]
for doc in preprocessed_docs:
    for w in doc:
        all_words.append(w)

In [14]:
len(all_words)

50008

In [35]:
frequency = {}
for word in all_words:
    count = frequency.get(word,0)
    frequency[word] = count + 1

In [36]:
# frequency = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
# print(type(frequency))

In [37]:
frequency['problem']

252

In [38]:
frequency_list = frequency.keys()

In [39]:
len(frequency)

5811

In [40]:
uniq_words=[]
for words in frequency_list:
    uniq_words.append(word)
    print( words, frequency[words])

year 109
tried 62
maybe 22
6 80
blu 342
phone 2864
looked 24
amazing 74
laggy 4
point 38
drove 3
nut 2
take 107
back 224
got 182
vivo 93
xl2 50
absolutely 45
happy 108
freaking 3
stylish 2
usual 5
ton 8
bell 7
whistle 7
like 339
expensive 32
far 159
work 453
amazingly 7
well 153
ca 108
nt 718
believe 25
nice 159
price 195
unreal 2
ignore 4
64 5
gig 7
sd 93
card 245
limitation 1
took 71
128 4
problem 252
love 296
new 239
keep 81
buying 48
blui 1
week 89
still 172
better 153
failed 6
mention 11
quick 29
charge 147
come 139
case 154
arrives 2
screen 414
protector 63
bubble 2
junk 12
film 2
also 157
extra 34
say 102
much 131
paid 22
people 55
get 294
riding 1
tech 12
high 34
amazon 104
shout 1
best 97
ever 74
obviously 12
open 32
honest 6
list 4
view 5
pro 10
con 11
phonepros 1
even 195
suggested 5
retail 5
149 6
usd 5
performs 3
tagbuild 1
quality 132
built 12
700 10
flagship 17
device 112
manages 1
look 141
beautiful 36
professionalcall 1
perfect 109
mean 32
supposed 14
doscreen 1
720p 5

32g 4
2g 3
simple 8
job 19
picked 6
spec 16
matched 1
definitely 59
minimal 6
viewable 1
clipboard 1
spoiled 1
flex 2
difference 28
fair 8
caller 4
lovin 1
grandkids 2
screenshot 1
perk 1
including 10
top 30
highly 27
crashed 4
spen 1
enjoying 5
hope 27
148 2
worst 27
niece 5
trinidad 2
noticed 7
overheated 1
mths 1
ring 8
laugh 1
touching 9
abroad 2
cut 12
loss 2
14999 2
learning 12
curve 5
switched 12
warning 2
rebooted 2
restarted 6
ended 15
wasted 5
hubby 1
activity 1
regret 15
punishment 1
performing 6
soldier 1
accident 4
handoff 1
concrete 3
thanks 24
visible 1
nick 1
suffered 2
partial 3
io 9
headbuds 1
14900 1
bargain 2
consistently 2
pleasing 2
eye 6
fashion 2
statement 3
consumer 5
claimed 2
competition 2
threat 2
function 28
advise 5
vibrates 2
exit 2
generally 4
became 6
googled 3
common 13
plan 23
145 1
recommending 4
800 4
gift 29
reconend 1
straight 20
gorgeous 9
aug 1
31 3
sept 2
caused 5
stolen 4
fan 9
rival 2
rear 11
envy 2
frequently 11
pricy 1
cphone 1
phablet 1
pr

moth 2
predecessor 2
torn 1
acquire 1
proved 3
checking 3
hesitate 1
terribly 2
fails 2
not•its 1
sensitive 7
extent 1
reacts 2
wire 1
closed 2
earphones•both 1
blurring 1
granulated 1
secondary 2
direct 1
snapped 1
indoors 2
undefined 1
photography 1
dramatic 1
•the 1
continuously 4
crash•but 1
department 3
counter 1
guessed 1
alive 1
exaggerationalso 1
forcefully 1
performanceso 1
resume 1
planning 4
reconsider 1
regretting 1
alternatively 1
downgrade 1
practically 2
receives 1
leading 2
penny 2
inside 6
malfunctioning 3
octacore 2
snappier 2
lmao 1
brazil 9
problems1 1
one2 1
isnot 1
stable 3
sometime 1
amazoncom 3
problem4 1
xl2i 1
exchange 8
quilty 1
desired 1
passcode 3
disconnected 1
thingthose 1
lying 4
repsi 1
offbrand 1
suxxxxxxhow 1
frame 6
privacy 1
unrelated 1
prodcut 1
failure 2
pitch 1
squealing 1
mini 3
tea 1
pot 1
inconvenientsometimes 1
reluctant 2
setup 5
wrote 2
joy 3
lip 4
yuk 1
output 2
component 3
questionable 2
corning 4
area 7
underneath 2
poorly 5
designed 2
c

contatec 1
suposed 1
chenag 1
prepaid 3
lagged 1
evades 1
tax 5
pretending 1
profit 2
cayman 1
island 2
northern 1
ireland 1
secret 1
court 1
150000 1
euro 2
french 1
protester 1
truth 1
expression 1
screw 3
america 4
california 1
evademy 1
jam 1
junked 1
market 12
userfriendly 3
twentyfive 1
itunes 1
recap 1
safest 1
endless 1
array 1
hi 2
kainn 1
host 1
appreciation 1
follower 1
ford190322 1
gmailcom 1
lasting 8
yelled 1
bent 1
laid 1
teetered 1
forth 2
cornerit 1
twisted 1
diagonally 1
argued 1
fragile 7
12500 1
19999 1
promte 3
pepole 1
pblm 1
chanel 1
thankyou 1
€ 1
baby 1
it🙏🏼💕 1
thumb 2
legit 1
belinda42 1
ugrading 1
phoneconcerns 1
skinny 1
lifeproof 2
cracking 1
grow 1
itpositives 1
apropriate 1
qualitly 1
beauty 2
bold 1
rugged 3
uag 2
babied 2
ithowever 1
trauma 1
cash 2
vertical 1
marketing 1
myou 1
2016 6
unexpectantly 1
camara 4
recueved 1
nope 2
countless 1
pls 1
marketfirst 1
uk 6
electrical 1
outlet 9
bulkiness 1
expectedanother 1
central 1
midwest 1
certain 1
ordinari

ideally 1
silently 1
wanders 1
aroundthe 1
grown 1
aesthetic 1
launchedthe 1
utilizing 1
chipset 1
820 4
marked 1
stutter 1
coupled 1
4gb 5
blaze 1
75 1
securewith 1
customizability 2
potentially 1
fleshed 1
monitored 1
adept/willing 1
tinkering 1
multi 1
multitask 2
calculator 1
modei 1
256gb 1
base 1
wellif 1
smartphonehavencom 1
brief 1
vibrancy 1
playing 6
better2 1
louder 1
audible 1
minispeaker 1
direction3 1
translates 1
nailed 1
efficiency 1
lollipop 2
usage4 1
downsized 1
12megapixel 1
16mpit 1
supposing 1
tree 1
backdrop 1
sharper 2
sky 1
utterly 3
camera5 1
winner6 1
sulk 1
swimming 2
pluscons 1
to2 1
s73 1
selfies4 1
inbuilt 1
outbox 1
problem5 1
readability 1
safari 1
conclusion 2
9/10verdict 1
gothamseller 1
46 1
concludes 1
wrapped 1
regretted 1
badlyto 1
frustratingmore 1
attitude 1
expressed 1
desire 1
resourcesgetting 1
inevitably 1
lunch 1
largely 1
unused 1
inexcusable 1
hike 2
packlastly 1
bizarre 1
phenomenon 1
describe 1
precision 1
— 1
borrow 1
fiveminute 1
capa

In [41]:
len(uniq_words)

5811

In [42]:
nltk.pos_tag(['Its', 'worth' ,'the' ,'wait' ,'getting', 'something', 'worth' ,'your', 'money'])

[('Its', 'PRP$'),
 ('worth', 'NN'),
 ('the', 'DT'),
 ('wait', 'NN'),
 ('getting', 'VBG'),
 ('something', 'NN'),
 ('worth', 'IN'),
 ('your', 'PRP$'),
 ('money', 'NN')]

In [43]:
pos_tags=[]

for i in preprocessed_docs:
    pos_tags.append(nltk.pos_tag(i)) 

In [44]:
pos_tags[1]

[('people', 'NNS'),
 ('love', 'VBP'),
 ('get', 'VB'),
 ('new', 'JJ'),
 ('phone', 'NN'),
 ('riding', 'VBG'),
 ('new', 'JJ'),
 ('tech', 'NN'),
 ('high', 'JJ'),
 ('get', 'NN'),
 ('amazon', 'VBN'),
 ('shout', 'RB'),
 ('best', 'JJS'),
 ('phone', 'NN'),
 ('ever', 'RB'),
 ('obviously', 'RB'),
 ('open', 'JJ'),
 ('honest', 'JJS'),
 ('list', 'NN'),
 ('view', 'NN'),
 ('pro', 'JJ'),
 ('con', 'NN'),
 ('phonepros', 'NN'),
 ('price', 'NN'),
 ('even', 'RB'),
 ('suggested', 'VBD'),
 ('retail', 'JJ'),
 ('149', 'CD'),
 ('usd', 'JJ'),
 ('phone', 'NN'),
 ('performs', 'NNS'),
 ('price', 'NN'),
 ('tagbuild', 'VBP'),
 ('quality', 'NN'),
 ('built', 'VBN'),
 ('like', 'IN'),
 ('700', 'CD'),
 ('usd', 'JJ'),
 ('flagship', 'NN'),
 ('device', 'NN'),
 ('manages', 'NNS'),
 ('look', 'VBP'),
 ('beautiful', 'JJ'),
 ('professionalcall', 'NN'),
 ('quality', 'NN'),
 ('perfect', 'VBP'),
 ('mean', 'VB'),
 ('phone', 'NN'),
 ('supposed', 'VBD'),
 ('doscreen', 'JJ'),
 ('720p', 'CD'),
 ('nice', 'JJ'),
 ('iphone', 'NN'),
 ('pixel'

In [45]:
noun_words=[]
verb_words=[]
adj_words=[]
for doc in pos_tags:
    for word_pos in doc :
        if word_pos[1] in ('NN','NNS','NNP','NNPS'):
            noun_words.append(word_pos[0]) 
        elif word_pos[1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
            verb_words.append(word_pos[0])
        elif word_pos[1] in ('JJR','JJS','JJ'):
            adj_words.append(word_pos[0])
        

In [46]:
print(len(set(adj_words)))

1813


In [47]:
noun_dict=dict((i, noun_words.count(i)) for i in noun_words)
noun_dict = sorted(noun_dict.items(), key=operator.itemgetter(1),reverse=True)
print((noun_dict))

[('phone', 2864), ('work', 421), ('time', 297), ('battery', 297), ('screen', 276), ('month', 254), ('problem', 252), ('blu', 249), ('camera', 246), ('card', 222), ('samsung', 222), ('use', 206), ('price', 195), ('product', 194), ('day', 192), ('issue', 171), ('nt', 160), ('case', 154), ('s7', 142), ('everything', 141), ('thing', 131), ('quality', 130), ('charge', 128), ('iphone', 127), ('service', 113), ('device', 111), ('year', 109), ('love', 109), ('money', 107), ('galaxy', 102), ('look', 101), ('life', 98), ('lot', 93), ('review', 92), ('call', 92), ('charger', 91), ('week', 89), ('get', 88), ('sim', 88), ('apps', 88), ('version', 87), ('feature', 86), ('picture', 84), ('purchase', 83), ('vivo', 79), ('hour', 79), ('memory', 77), ('box', 72), ('brand', 71), ('customer', 71), ('’', 68), ('way', 68), ('buy', 67), ('return', 67), ('amazon', 66), ('condition', 65), ('seller', 64), ('protector', 62), ('apple', 62), ('storage', 60), ('glass', 60), ('button', 58), ('nothing', 58), ('water'

In [48]:
verb_dict=dict((i, verb_words.count(i)) for i in verb_words)
verb_dict = sorted(verb_dict.items(), key=operator.itemgetter(1),reverse=True)
print((verb_dict))

[('nt', 223), ('get', 204), ('bought', 183), ('got', 177), ('love', 166), ('working', 150), ('came', 146), ('buy', 135), ('using', 126), ('used', 109), ('go', 105), ('come', 101), ('say', 101), ('make', 97), ('take', 92), ('received', 87), ('charging', 82), ('worked', 81), ('know', 77), ('started', 77), ('getting', 76), ('want', 75), ('took', 71), ('keep', 65), ('recommend', 64), ('going', 62), ('went', 59), ('made', 58), ('purchased', 58), ('need', 57), ('stopped', 57), ('expected', 56), ('put', 51), ('think', 51), ('tried', 48), ('see', 48), ('amazing', 47), ('seems', 47), ('said', 46), ('told', 42), ('give', 41), ('looking', 39), ('loved', 39), ('refurbished', 39), ('dropped', 38), ('found', 38), ('use', 38), ('replace', 37), ('turn', 37), ('look', 35), ('find', 35), ('called', 34), ('set', 34), ('run', 33), ('needed', 33), ('buying', 31), ('setting', 31), ('send', 31), ('work', 31), ('wanted', 30), ('thought', 30), ('sent', 30), ('unlocked', 29), ('arrived', 28), ('owned', 27), ('g

In [49]:
adj_dict=dict((i, adj_words.count(i)) for i in adj_words)
adj_dict = sorted(adj_dict.items(), key=operator.itemgetter(1),reverse=True)
print((adj_dict))

[('great', 568), ('good', 474), ('new', 239), ('nt', 206), ('nice', 138), ('screen', 136), ('last', 127), ('happy', 108), ('much', 88), ('excellent', 88), ('bad', 85), ('u', 85), ('best', 77), ('sim', 76), ('little', 74), ('perfect', 67), ('update', 65), ('old', 62), ('international', 62), ('easy', 60), ('second', 60), ('samsung', 59), ('unlocked', 56), ('big', 55), ('many', 53), ('fine', 53), ('first', 53), ('able', 52), ('better', 51), ('different', 50), ('android', 50), ('overall', 49), ('awesome', 49), ('blu', 48), ('sure', 47), ('sd', 44), ('original', 40), ('hard', 39), ('apps', 39), ('several', 38), ('long', 37), ('fast', 37), ('previous', 36), ('low', 36), ('free', 35), ('s7', 35), ('high', 34), ('full', 34), ('open', 32), ('beautiful', 32), ('slow', 32), ('right', 32), ('worth', 32), ('extra', 31), ('black', 31), ('clear', 31), ('small', 30), ('expensive', 29), ('due', 29), ('least', 28), ('le', 28), ('disappointed', 27), ('enough', 27), ('upgrade', 27), ('couple', 27), ('real

In [50]:
imp_nouns=[]
imp_verbs=[]
imp_adjs=[]
for k,v in noun_dict:
    if(v>=5):
        imp_nouns.append(k)
for k,v in verb_dict:
    if(v>=5):
        imp_verbs.append(k)
for k,v in adj_dict:
    if(v>=5):
        imp_adjs.append(k)
        

In [51]:
print(len(imp_nouns),len(imp_verbs),len(imp_adjs))

629 354 334


In [52]:
print(imp_nouns)

['phone', 'work', 'time', 'battery', 'screen', 'month', 'problem', 'blu', 'camera', 'card', 'samsung', 'use', 'price', 'product', 'day', 'issue', 'nt', 'case', 's7', 'everything', 'thing', 'quality', 'charge', 'iphone', 'service', 'device', 'year', 'love', 'money', 'galaxy', 'look', 'life', 'lot', 'review', 'call', 'charger', 'week', 'get', 'sim', 'apps', 'version', 'feature', 'picture', 'purchase', 'vivo', 'hour', 'memory', 'box', 'brand', 'customer', '’', 'way', 'buy', 'return', 'amazon', 'condition', 'seller', 'protector', 'apple', 'storage', 'glass', 'button', 'nothing', 'water', 'support', 'speaker', 'software', 'people', 'color', 'app', 'pay', 'carrier', 'warranty', 'store', 'edge', 'reason', 'hand', 'size', 'replacement', 'deal', 'something', 'cell', 'home', 'power', 'smartphone', 'gold', 'slot', 'anything', 'xl', 'display', 'data', 'change', 'feel', 'star', 'speed', 'item', 'model', 'video', 'point', 'xl2', 'help', 'waste', 'android', 'turn', 's6', 'today', 'complaint', 'note',

In [54]:
lemmatised_docs=[]
for i in preprocessed_docs:
    str=' '.join(i)
    lemmatised_docs.append(str)
    

In [55]:
lemmatised_docs[5]

'give 5 star price phenomenal deal waited 2 week write review must say great phone look good work really well storage terrific download evie go launcher nt like stock launcher never spending 700 phone anymore long blu make affordable awesome device like vivo xl2'

https://buhrmann.github.io/tfidf-analysis.html

In [63]:
def top_tfidf_feats(row, features,top_n):
    #top_n=25
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [64]:
def top_feats_in_doc(Xtr, features, row_id,top_n):
    #top_n=25
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features,top_n)

In [65]:
def top_mean_feats(Xtr, features,top_n):
    grp_ids=None
    min_tfidf=0.1
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features,top_n)

In [66]:
def top_feats_by_class(Xtr, y, features,top_n):
    min_tfidf=0.1
    
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [67]:
top_feats_in_doc(tfidf,tfidf_features,5,5)

,feature,tfidf
0,launcher,0.373807
1,phenomenal,0.239796
2,waited,0.226471
3,terrific,0.226471
4,affordable,0.217017


In [68]:
top_mean_feats(tfidf,tfidf_features,50)

,feature,tfidf
0,phone,0.079864
1,great,0.049937
2,good,0.048510
3,love,0.035554
4,work,0.031353
5,nt,0.025611
6,excellent,0.023788
7,product,0.022924
8,perfect,0.018530
9,like,0.018089


In [74]:
print(tf.shape)

(2170, 2547)


In [75]:
tf.data

array([1, 1, 1, ..., 2, 1, 1], dtype=int64)